<a href="https://colab.research.google.com/github/dhanush852/intro_to-ml/blob/main/HOME_WORK_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.0 MB/s eta 0:00:00
time: 413 µs (started: 2023-12-13 01:09:53 +00:00)


In [3]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)
np.random.seed(123)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "CPU")
device



device(type='cuda')

In [6]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"




In [7]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
imgs = torch.stack([img for img, _ in train_dataset], dim=3)
mean, std = imgs.view(3, -1).mean(dim=1), imgs.view(3, -1).std(dim=1)


100%|██████████| 170498071/170498071 [00:03<00:00, 47625515.75it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [8]:
mean, std

(tensor([0.4914, 0.4822, 0.4465]), tensor([0.2470, 0.2435, 0.2616]))

In [9]:


cifar10 = datasets.CIFAR10('./data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [12]:
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [11]:
trainLoad = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
testLoad = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)
classNames = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [13]:
import datetime

In [14]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 2 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [15]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [16]:
import torch.nn.functional as F

In [17]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out



In [20]:
model = NetResDeep(n_chans1=32, n_blocks=10).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()
training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = trainLoad,
)

2023-12-13 01:27:03.845440 Epoch 1, Training loss 1.6408428361762126
2023-12-13 01:27:35.494973 Epoch 2, Training loss 1.3125485861751411
2023-12-13 01:28:38.287586 Epoch 4, Training loss 1.0913012855646325
2023-12-13 01:29:42.265201 Epoch 6, Training loss 0.9693882665951437
2023-12-13 01:30:45.141597 Epoch 8, Training loss 0.8891217424102266
2023-12-13 01:31:49.668626 Epoch 10, Training loss 0.8263548747782362
2023-12-13 01:32:52.954891 Epoch 12, Training loss 0.7709218527900051
2023-12-13 01:33:57.595598 Epoch 14, Training loss 0.7286793912212131
2023-12-13 01:35:00.809796 Epoch 16, Training loss 0.6925601780757794
2023-12-13 01:36:05.758714 Epoch 18, Training loss 0.6614179966240759
2023-12-13 01:37:08.994209 Epoch 20, Training loss 0.6252877299345538
2023-12-13 01:38:12.631133 Epoch 22, Training loss 0.5959548180874006
2023-12-13 01:39:16.895220 Epoch 24, Training loss 0.5688352908221713
2023-12-13 01:40:20.730564 Epoch 26, Training loss 0.5410055098477191
2023-12-13 01:41:23.29732

In [21]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
def validate(model, train_loader, val_loader):
    accdict = {}
    predictions = []
    exp_labels = []

    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

                predictions.extend(predicted.cpu().numpy())
                exp_labels.extend(labels.cpu().numpy())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict, predictions, exp_labels


In [22]:
accuracy, predictions, expected_labels = validate(model, train_loader, val_loader)


Accuracy train: 0.95
Accuracy val: 0.66


In [23]:
precision_d = precision_score(predictions, expected_labels, average='macro')
recall_d = recall_score(predictions, expected_labels, average='macro')
cnfMatrix = confusion_matrix(predictions, expected_labels)
print(cnfMatrix)

In [26]:
print(classification_report(predictions, expected_labels, target_names=classNames))

              precision    recall  f1-score   support

    airplane       0.96      0.81      0.88      7098
  automobile       0.93      0.94      0.94      5968
        bird       0.85      0.92      0.88      5560
         cat       0.89      0.82      0.85      6550
        deer       0.89      0.91      0.90      5887
         dog       0.83      0.94      0.88      5333
        frog       0.93      0.94      0.94      5919
       horse       0.95      0.88      0.92      6458
        ship       0.96      0.91      0.94      6306
       truck       0.80      0.98      0.88      4921

    accuracy                           0.90     60000
   macro avg       0.90      0.91      0.90     60000
weighted avg       0.91      0.90      0.90     60000



In [27]:
class NetWidth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [28]:

def training_loop_l2reg(n_epochs, optimizer, model, loss_fn,
                        train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())  # <1>
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


In [29]:

modelWd = NetWidth(n_chans1=32).to(device=device)
optimizer_wd = optim.SGD(modelWd.parameters(), lr=1e-2)
lossFn = nn.CrossEntropyLoss()


training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer_wd,
    model = modelWd,
    loss_fn = lossFn,
    train_loader = train_loader,
)


2023-12-13 02:22:06.371203 Epoch 1, Training loss 2.0321958574187726
2023-12-13 02:24:39.956756 Epoch 10, Training loss 1.146674206952
2023-12-13 02:27:29.264053 Epoch 20, Training loss 0.9690431043162675
2023-12-13 02:30:20.476496 Epoch 30, Training loss 0.8994943727465237
2023-12-13 02:33:10.151513 Epoch 40, Training loss 0.8590997023808072
2023-12-13 02:35:59.973098 Epoch 50, Training loss 0.831738175104951
2023-12-13 02:38:49.145809 Epoch 60, Training loss 0.811712636819581
2023-12-13 02:41:38.891280 Epoch 70, Training loss 0.7966082192138028
2023-12-13 02:44:27.072983 Epoch 80, Training loss 0.7848985903250897
2023-12-13 02:47:16.535805 Epoch 90, Training loss 0.7753838341864173
2023-12-13 02:50:05.235128 Epoch 100, Training loss 0.7674891792447366


In [31]:
accuracyWd, predictionsWd, expected_labels_wd = validate(modelWd, train_loader, val_loader)
precisionWd = precision_score(predictionsWd, expected_labels_wd, average='macro')
recallWd = recall_score(predictionsWd, expected_labels_wd, average='macro')
cnf_matrixWd = confusion_matrix(predictionsWd, expected_labels_wd)
print(cnf_matrixWd)

Accuracy train: 0.81
Accuracy val: 0.70
[[5074   85  306  114  155   49   40   52  539  159]
 [ 127 5489   14   41   19   17   36   23  238  375]
 [ 249   48 4142  283  342  224  224  144  105   44]
 [ 126   41  340 4192  362  887  425  209   99   70]
 [  68   16  446  214 4370  188  192  194   32   25]
 [  44   20  259  749  204 4303  115  234   34   25]
 [  13   18  263  204  206   88 4916   13   22   24]
 [  66   25  164  152  297  221   17 5100   25   84]
 [ 111   37   29   28   20    8   18    8 4770   64]
 [ 122  221   37   23   25   15   17   23  136 5130]]


In [33]:
print(classification_report(predictionsWd, expected_labels_wd, target_names=classNames))


              precision    recall  f1-score   support

    airplane       0.85      0.77      0.81      6573
  automobile       0.91      0.86      0.89      6379
        bird       0.69      0.71      0.70      5805
         cat       0.70      0.62      0.66      6751
        deer       0.73      0.76      0.74      5745
         dog       0.72      0.72      0.72      5987
        frog       0.82      0.85      0.84      5767
       horse       0.85      0.83      0.84      6151
        ship       0.80      0.94      0.86      5093
       truck       0.85      0.89      0.87      5749

    accuracy                           0.79     60000
   macro avg       0.79      0.80      0.79     60000
weighted avg       0.79      0.79      0.79     60000



In [34]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [35]:
modelDropout = NetDropout(n_chans1=32).to(device=device)
optimizerDropout = optim.SGD(modelDropout.parameters(), lr=1e-2)
lossFn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizerDropout,
    model = modelDropout,
    loss_fn = lossFn,
    train_loader = train_loader,
)


2023-12-13 02:58:26.673233 Epoch 1, Training loss 2.0356376956186026
2023-12-13 02:58:43.075073 Epoch 2, Training loss 1.777426349842335
2023-12-13 02:59:16.257933 Epoch 4, Training loss 1.5672250080596455
2023-12-13 02:59:47.929598 Epoch 6, Training loss 1.4729617579513803
2023-12-13 03:00:40.138092 Epoch 8, Training loss 1.3992077950626383
2023-12-13 03:01:19.349908 Epoch 10, Training loss 1.338284488071871
2023-12-13 03:02:12.076162 Epoch 12, Training loss 1.2926866944945987
2023-12-13 03:02:53.521293 Epoch 14, Training loss 1.2583493949354763
2023-12-13 03:03:30.665549 Epoch 16, Training loss 1.226048232801735
2023-12-13 03:04:15.837204 Epoch 18, Training loss 1.2041681144396057
2023-12-13 03:05:07.356227 Epoch 20, Training loss 1.181639991376711
2023-12-13 03:05:50.726448 Epoch 22, Training loss 1.1565201661318465
2023-12-13 03:06:25.111282 Epoch 24, Training loss 1.141395692706413
2023-12-13 03:07:00.172785 Epoch 26, Training loss 1.1272767575653009
2023-12-13 03:07:33.052962 Epo

In [36]:
accuracyDropout, predictionsDropout, expected_labels_dropout = validate(modelDropout, train_loader, val_loader)

Accuracy train: 0.67
Accuracy val: 0.62


In [37]:
precisionDropout = precision_score(predictionsDropout, expected_labels_dropout, average='macro')
recallDropout = recall_score(predictionsDropout, expected_labels_dropout, average='macro')
cnf_matrixDropout = confusion_matrix(predictionsDropout, expected_labels_dropout)
print(cnf_matrixDropout)

[[4264  133  415  117  174   53   41   75  558  201]
 [ 169 4715   63   81   35   44   46   42  296  723]
 [ 309   48 3139  395  519  386  257  237  122   54]
 [ 185   98  449 3098  452 1162  504  348  126  130]
 [ 135   37  576  348 3492  319  290  395   58   51]
 [  81   32  427  988  290 3305  147  492   49   76]
 [  78   77  502  553  526  251 4567  117   69   84]
 [ 106   52  266  242  410  363   53 4143   46  155]
 [ 414  170  111   81   71   47   51   38 4454  166]
 [ 259  638   52   97   31   70   44  113  222 4360]]


In [38]:
print(classification_report(predictionsDropout, expected_labels_dropout, target_names=classNames))

              precision    recall  f1-score   support

    airplane       0.71      0.71      0.71      6031
  automobile       0.79      0.76      0.77      6214
        bird       0.52      0.57      0.55      5466
         cat       0.52      0.47      0.49      6552
        deer       0.58      0.61      0.60      5701
         dog       0.55      0.56      0.56      5887
        frog       0.76      0.67      0.71      6824
       horse       0.69      0.71      0.70      5836
        ship       0.74      0.79      0.77      5603
       truck       0.73      0.74      0.73      5886

    accuracy                           0.66     60000
   macro avg       0.66      0.66      0.66     60000
weighted avg       0.66      0.66      0.66     60000



In [39]:

class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features=n_chans1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features=n_chans1 // 2)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.conv1_batchnorm(self.conv1(x))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = self.conv2_batchnorm(self.conv2(out))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [40]:
model_batch_norm = NetBatchNorm(n_chans1=32).to(device=device)
optimizer_batch_norm = optim.SGD(model_batch_norm.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer_batch_norm,
    model = model_batch_norm,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


2023-12-13 03:30:01.004776 Epoch 1, Training loss 1.7788722178210383
2023-12-13 03:30:17.505987 Epoch 2, Training loss 1.4811666915788675
2023-12-13 03:30:51.403177 Epoch 4, Training loss 1.257535622400396
2023-12-13 03:31:24.814614 Epoch 6, Training loss 1.1194834946214085
2023-12-13 03:31:59.029620 Epoch 8, Training loss 1.0272434318004666
2023-12-13 03:32:32.247301 Epoch 10, Training loss 0.9607238463123741
2023-12-13 03:33:06.013086 Epoch 12, Training loss 0.9091478647935726
2023-12-13 03:33:39.238659 Epoch 14, Training loss 0.8679635697008704
2023-12-13 03:34:13.152799 Epoch 16, Training loss 0.8341454881079057
2023-12-13 03:34:47.004540 Epoch 18, Training loss 0.8052948688911965
2023-12-13 03:35:20.588938 Epoch 20, Training loss 0.7799882767221812
2023-12-13 03:35:54.304709 Epoch 22, Training loss 0.7573165939287152
2023-12-13 03:36:27.159817 Epoch 24, Training loss 0.7367510805883066
2023-12-13 03:37:00.751721 Epoch 26, Training loss 0.7178838554474399
2023-12-13 03:37:33.060604

In [41]:
accuracy_batchNorm, predictions_batchNorm, expected_labels_batchNorm = validate(modelDropout, train_loader, val_loader)

Accuracy train: 0.67
Accuracy val: 0.61


In [42]:
precision_batchNorm = precision_score(predictions_batchNorm, expected_labels_batchNorm, average='macro')
recall_batchNorm = recall_score(predictions_batchNorm, expected_labels_batchNorm, average='macro')
cnf_matrix_batchNorm = confusion_matrix(predictions_batchNorm, expected_labels_batchNorm)
print(cnf_matrix_batchNorm)

[[4257  127  420  125  183   58   42   60  562  207]
 [ 178 4709   36   80   28   49   60   41  307  692]
 [ 318   48 3179  397  575  376  270  231  114   47]
 [ 179   74  474 3131  454 1158  445  350  133  160]
 [ 114   43  565  326 3488  315  281  439   71   48]
 [  78   33  426  977  244 3327  151  467   48   65]
 [  65   91  475  550  524  276 4588  123   62   78]
 [ 105   40  255  220  397  347   68 4127   47  182]
 [ 426  183  118   76   66   45   50   39 4416  151]
 [ 280  652   52  118   41   49   45  123  240 4370]]


In [43]:
print(classification_report(predictionsDropout, expected_labels_dropout, target_names=classNames))

              precision    recall  f1-score   support

    airplane       0.71      0.71      0.71      6031
  automobile       0.79      0.76      0.77      6214
        bird       0.52      0.57      0.55      5466
         cat       0.52      0.47      0.49      6552
        deer       0.58      0.61      0.60      5701
         dog       0.55      0.56      0.56      5887
        frog       0.76      0.67      0.71      6824
       horse       0.69      0.71      0.70      5836
        ship       0.74      0.79      0.77      5603
       truck       0.73      0.74      0.73      5886

    accuracy                           0.66     60000
   macro avg       0.66      0.66      0.66     60000
weighted avg       0.66      0.66      0.66     60000

